# **1. Preprocesamiento** **de** **los** **datos** **de** **entrenamiento** **de** **los** **modelos** **NLP**
En este módulo se lleva a cabo el preprocesamiento de los datos que servirán como entrada para el entrenamiento de los modelos de Procesamiento de Lenguaje Natural (PLN) utilizados en nuestra aplicación.

La fuente primaria de datos está conformada por recetas de cocina extraídas del portal elmundo.es, lo que garantiza un corpus extenso y heterogéneo en idioma español.

El pipeline de preprocesamiento comprende las siguientes etapas:

1. Extracción de datos mediante  scraping automatizado de recetas directamente desde la web.

2. Preprocesamiento y enriquecimiento: Limpieza, normalización y estructuración de los textos obtenidos.

3. Generación de dataset etiquetado: Utilización de prompting mediante la API de OpenAI para obtener conjuntos de datos con anotaciones específicas orientadas a tareas de clasificación y reconocimiento de entidades nombradas (NER).

In [ ]:
!pip install requests beautifulsoup4 pandas
!pip install -q openai ipywidgets python-dotenv
!pip install unidecode

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 36.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 6.8 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from openai import OpenAI
import os
import json

El flujo de preprocesamiento será el siguiente:

1)Scrapping de todas las páginas que contengan recetas. Debido a que en la web hay entradas que no contienen recetas en sí, se considerará como receta aquella que contenga una lista de ingredientes

2)Prompting de la página para obtener de ella una lista de ingredientes y una lista de pasos de la receta

3)Procesamiento de la lista de pasos: con ello se obtiene un texto unificado de la receta a partir de la secuencia de pasos de la misma.

4)Definición de los spans de las distintas etiquetas(ingredientes) de cada receta.

6)Obtención del dataset de entrenamiento, en el cual cada receta tiene un texto con la receta y la lista de spans


In [ ]:
os.environ["OPENAI_API_KEY"] = ""
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])

No todas las entradas de la webse corresponden con recetas. Algunas entradas contienen consejos de cocina, etc. Para extraer aquellas entradas que sí se corresponden con recetas, se ha observado un patrón que contienen todas estas: un listado de ingredientes. Con la función ***extraer_ingredientes*** extraemos esta lista de ingredientes, lo cual nos permitirá evaluar si la página es de una receta o no.

In [ ]:
#Función que extrae de la web la lista de ingredientes, de haberla. Necesaria para filtrar aquellas webs que nos sirven de las que no
def extraer_ingredientes(url):
    try:
        response = requests.get(url)
        if response.status_code != 200:
            return []

        soup = BeautifulSoup(response.content, "html.parser")

        # Buscar cualquier etiqueta que contenga "ingredientes"
        posibles_encabezados = [
            tag for tag in soup.find_all(True)
            if tag.get_text(strip=True).lower().startswith("ingredientes")
        ]

        ingredientes = []

        for encabezado in posibles_encabezados:
            bloque = encabezado.find_next("ul")
            if bloque:
                ingredientes += [li.get_text(strip=True) for li in bloque.find_all("li")]

        return ingredientes
    except Exception as e:
        print("Error:", e)
        return []

La siguiente función sirve para procesar mediante un prompt a Open AI las recetas de la web. La receta procesada consistira en un diccionario con las siguientes claves:

- Título de la receta

- URL

-Ingredientes que aparecen en la receta (etiquetas que usaremos posteriormente en el modelo NER)


-Pasos de la receta (nos servirá para entrenar el modelo de clasificación)



In [ ]:
#Función procesamiento de las recetas, obteniendo de cada una de ellas: titulo, url, ingredientes y pasos
def procesar_receta_con_gpt(url, max_chars_html=20000):
    try:
        r = requests.get(url, timeout=20)
        if r.status_code != 200:
            print(f"Error al acceder a {url}: {r.status_code}")
            return None

        # Texto plano del HTML (recorta para no mandar un libro al modelo)
        texto_receta = BeautifulSoup(r.content, "html.parser").get_text(" ", strip=True)
        if len(texto_receta) > max_chars_html:
            texto_receta = texto_receta[:max_chars_html]

        prompt = f"""
Responde SOLO con un JSON válido, sin texto extra.

Esquema exacto:
{{
  "ingredientes": ["...", "...", "..."],
  "pasos": ["...", "..."]
}}

Reglas:
- "ingredientes": lista deduplicada de NOMBRES (solo sustantivos) en minúsculas, sin cantidades, unidades, marcas ni adjetivos.
  Normalización:
  * Recorta espacios, elimina tildes para comparar, devuelve con tildes si aparecen en la fuente (p. ej., “azúcar”).
  * Singulariza plurales simples (“tomates”→“tomate”, “cebollas”→“cebolla”).
  * Compósitos: conserva la forma más informativa en el orden canónico (p. ej., “aceite de oliva”, “vinagre de manzana”, “azúcar glas”).
  * Quita calificativos/adjetivos no nucleares (“grande”, “picada”, “fresco”, “virgen extra”) salvo que definan el compuesto (“azúcar glas”, “leche evaporada”).
  * Unidades/cantidades fuera (“2 cda”, “200 g”, “1/2 taza”, “ml”, “kg”, “c/s”).
  * Sinónimos/abreviaturas comunes → forma canónica: “aove”→“aceite de oliva”; “azúcar glass/azúcar en polvo”→“azúcar glas”; “bicarbonato sódico”→“bicarbonato”.
  * Deduplica tras normalizar.
  *Incluye TODOS los ingredientes que se detecten en la receta, aunque no estén en la lista de ingredientes** (por ejemplo, mencionados solo en los pasos).
  Orden: por la PRIMERA aparición del ingrediente en los “pasos” (índices 1..N).

- Detección y agrupado de pasos:
  * Si la receta trae numeración explícita (p. ej., "Paso 1", "1.", "1)", "1.-"):
    - Conserva TODOS los pasos en el MISMO orden de la fuente. NO renumeres.
    - Todo el texto CONTIGUO desde un marcador de paso hasta el siguiente marcador pertenece al MISMO paso. No lo dividas aunque tenga varias frases o párrafos; únelas en un único string limpio.
    - Puede haber pasos que no mencionen ingredientes; se mantienen igualmente.
  * Si NO hay numeración explícita, segmenta en pasos breves por orden lógico y numéralos 1..N.

- "pasos": lista ordenada de textos de paso (una frase breve por paso). Si hay numeración explícita, debe corresponder 1:1 con los pasos originales.
Ejemplo:
Texto: "Ingredientes:Cebolla. Paso 1: Corta la cebolla. Paso 2: Mezcla la cebolla con el tomate. Paso 3: Sirve."
Salida:
{{
  "ingredientes": ["cebolla","tomate"],
  "pasos": ["Corta la cebolla.","Mezcla la cebolla con el tomate.","Sirve."],
}}

Texto de la receta:
\"\"\"{texto_receta}\"\"\"
""".strip()

        resp = client.chat.completions.create(
            model="gpt-4o-mini",
            temperature=0,
            messages=[
                {"role": "system", "content": "Eres estricto: solo devuelves JSON válido exactamente como se solicita."},
                {"role": "user", "content": prompt},
            ],
        )

        content = resp.choices[0].message.content  # <- clave del bug

        # Intentar parsear JSON directamente
        try:
            data = json.loads(content)
            return data
        except json.JSONDecodeError:
            # Intento de extracción de bloque JSON si vino con texto extra
            match = re.search(r"\{.*\}", content, flags=re.DOTALL)
            if match:
                try:
                    return json.loads(match.group(0))
                except json.JSONDecodeError:
                    pass
            print("No se pudo parsear JSON desde la salida del modelo.")
            return None

    except Exception as e:
        print("Error en procesar_receta_con_gpt:", e)
        return None

Antes de aplicar el pipeline de scrapping + procesamiento por prompting de las recetas, necesitamos definir esta la función auxiliar *normalizar_url* que Nos servirá para no procesar mas de una vez la misma url,ya el scrapping se hace recorriendo página a página la web y existen entradas que se repiten en distintas páginas (por ejemplo, la entrada "gazpacho" podría encontrarse en la página "Recetas españolas" y en la páhgina "recetas de verano", por lo que si no usamos esta función tendríamos dos recetas idénticas repetidas en nuestro dataset de entrenamiento.

In [ ]:
#Funcion para normalizar urls. Nos servirá para no procesar mas de una vez la misma url
from urllib.parse import urlsplit, urlunsplit

def normalizar_url(u: str) -> str:
    s = urlsplit(u.strip())
    scheme = (s.scheme or "https").lower()
    netloc = s.netloc.lower()
    path = s.path.rstrip("/")
    return urlunsplit((scheme, netloc, path, "", ""))  # sin query ni fragmento

Con las funciones anteriormente descritas aplicamos un pipeline a las siguientes páginas de la web:

- Recetas faciles

- Recetas rapidas

- Recetas sanas

- Recetas pollo

- Recetas pasta

- Recetas carne

- Recetas pescado

- Recetas verdura

Obteniendo un corpus inicial de 1189 recetas.



In [ ]:
# Primer set de recetas
base_url = "https://recetasdecocina.elmundo.es/recetas/faciles"
recetas = []
max_paginas = 10
vistas_url = set()

for page in range(1, max_paginas + 1):
    url = base_url if page == 1 else f"{base_url}/page/{page}"
    print(f" Visitando: {url}")

    response = requests.get(url)
    if response.status_code != 200:
        print(f" Error al acceder a la página {page}")
        break

    soup = BeautifulSoup(response.content, "html.parser")
    enlaces = soup.find_all("a", attrs={"rel": "bookmark"})
    if not enlaces:
        print(" No se encontraron recetas en esta página. Fin.")
        break

    nuevas = 0
    for a in enlaces:
        href = a.get("href")
        title = a.get("title")
        if not href or not title:
            continue

        href_norm = normalizar_url(href)
        if href_norm in vistas_url:
            continue  # ya la procesaste


        ingredientes_previos = extraer_ingredientes(href_norm)
        if not ingredientes_previos:
          continue

        datos_gpt = procesar_receta_con_gpt(href_norm)
        if not datos_gpt:
            continue

        receta = {
            "titulo": title.strip(),
            "url": href_norm,  # guarda la URL normalizada
            "ingredientes": datos_gpt.get("ingredientes", []),
            "pasos": datos_gpt.get("pasos", [])
        }
        recetas.append(receta)
        vistas_url.add(href_norm)
        nuevas += 1

    print(f" Se añadieron {nuevas} recetas con ingredientes.\n")
    if nuevas == 0:
        print(" No hay recetas nuevas con ingredientes. Fin del scraping.")
        break

 Visitando: https://recetasdecocina.elmundo.es/recetas/faciles
 Se añadieron 20 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/faciles/page/2
 Se añadieron 20 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/faciles/page/3
 Se añadieron 19 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/faciles/page/4
 Se añadieron 20 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/faciles/page/5
 Se añadieron 19 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/faciles/page/6
 Se añadieron 19 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/faciles/page/7
 Se añadieron 12 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/faciles/page/8
 Error al acceder a la página 8


In [ ]:
# Sucesivos set de recetas
base_url = "https://recetasdecocina.elmundo.es/recetas/verduras"
max_paginas = 45


for page in range(1, max_paginas + 1):
    url = base_url if page == 1 else f"{base_url}/page/{page}"
    print(f" Visitando: {url}")

    response = requests.get(url)
    if response.status_code != 200:
        print(f" Error al acceder a la página {page}")
        break

    soup = BeautifulSoup(response.content, "html.parser")
    enlaces = soup.find_all("a", attrs={"rel": "bookmark"})
    if not enlaces:
        print(" No se encontraron recetas en esta página. Fin.")
        break

    nuevas = 0
    for a in enlaces:
        href = a.get("href")
        title = a.get("title")
        if not href or not title:
            continue

        href_norm = normalizar_url(href)
        if href_norm in vistas_url:
            continue  # ya la procesaste


        ingredientes_previos = extraer_ingredientes(href_norm)
        if not ingredientes_previos:
          continue

        datos_gpt = procesar_receta_con_gpt(href_norm)
        if not datos_gpt:
            continue

        receta = {
            "titulo": title.strip(),
            "url": href_norm,  # guarda la URL normalizada
            "ingredientes": datos_gpt.get("ingredientes", []),
            "pasos": datos_gpt.get("pasos", [])
        }
        recetas.append(receta)
        vistas_url.add(href_norm)
        nuevas += 1

    print(f" Se añadieron {nuevas} recetas con ingredientes.\n")
    if nuevas == 0:
        print(" No hay recetas nuevas con ingredientes. Fin del scraping.")
        break

 Visitando: https://recetasdecocina.elmundo.es/recetas/verduras
 Se añadieron 1 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/verduras/page/2
 Se añadieron 1 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/verduras/page/3
 Se añadieron 1 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/verduras/page/4
 Se añadieron 4 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/verduras/page/5
 Se añadieron 1 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/verduras/page/6
 Se añadieron 3 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/verduras/page/7
 Se añadieron 4 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/verduras/page/8
 Se añadieron 1 recetas con ingredientes.

 Visitando: https://recetasdecocina.elmundo.es/recetas/verduras/page/9
 Se añadieron 3 recetas 

In [ ]:
len(recetas)

1189

Guardamos el recetario procesado en formato json

In [ ]:
from pathlib import Path
#guardamos la lista con las recetas procesadas
Path("recetas.json").write_text(
    json.dumps(recetas, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

In [ ]:
# Leerlo de vuelta
contenido = Path("recetas.json").read_text(encoding="utf-8")
recetas_cargadas = json.loads(contenido)

Como se ha mencionado anteriormente, el corpus es una lista de diccionarios en la que cada diccionario corresponde una receta. A modo de ejemplo se muestra una entrada de la lista, correspondiente a la receta "Pochas copn pollo de corral"

In [ ]:
recetas_cargadas[752]

{'titulo': 'Pochas con pollo de corral, una receta sana y casera',
 'url': 'https://recetasdecocina.elmundo.es/2021/01/pochas-pollo-corral-receta-sana-casera.html',
 'ingredientes': ['pollo',
  'ajo',
  'pimiento rojo',
  'pimiento verde',
  'cebolleta',
  'tomate frito',
  'pimentón',
  'zanahoria',
  'pochas',
  'azafrán',
  'caldo',
  'sal'],
 'pasos': ['Comenzamos haciendo el sofrito. Pochamos las verduras cortadas muy finamente. Durante unos 20 minutos a fuego lento. Cuando tengamos las verduras pochadas ponemos unas hebras de azafrán y un poco de pimentón que le van a dar un toque espectacular.',
  'Mientras tanto en una sartén, doramos los trozos. Previamente los hemos tenido que salpimentar. Una vez dorados, retiramos y reservamos.',
  'Cuando este el sofrito, agregamos el pollo dorado, las pochas frescas y cubrimos con caldo de pollo.',
  'Cocinamos durante unos 20 minutos a fuego lento.']}

Nuestro objetivo con el modelo de clasificación de oraciones es agrupar las oraciones en pasos, identificando en ellas patrones que permitan al modelo discernir si una oración pertenece a un nuevo paso o es parte de un paso anterior. Para entrenar este modelo, el input propuesto para cada receta es un texto que contenga la receta de seguido, sin pasos. A este texto le llamaremos "resumen" y será una nueva entrada del diccionario de cada receta. Para obtenerlo definimos la función ***agregar_resumen_a_receta***.

In [ ]:
#Funcion para añadir resumen a la receta
import re
from typing import Dict, List, Any

def agregar_resumen_a_receta(receta: Dict[str, Any], clave_pasos: str = "pasos", clave_resumen: str = "resumen") -> Dict[str, Any]:
    """
    Añade receta[clave_resumen] uniendo receta[clave_pasos] (lista de strings)
    en una sola cadena, separadas por punto. Limpia puntuación final duplicada.

    - Si no hay pasos o no es una lista de strings, pone resumen = "".
    - Devuelve el propio diccionario (mutado) por conveniencia.
    """
    pasos = receta.get(clave_pasos) or []
    if not isinstance(pasos, list):
        receta[clave_resumen] = ""
        return receta

    limpios: List[str] = []
    for p in pasos:
        if not isinstance(p, str):
            continue
        s = p.strip()
        # quita puntuación final para evitar ".."
        s = re.sub(r"\s*([.;:!?…]+)\s*$", "", s)
        if s:
            limpios.append(s)

    # une con ". " y añade punto final si hay contenido
    resumen = ". ".join(limpios)
    if resumen:
        resumen += "."
    receta[clave_resumen] = resumen
    return receta

In [ ]:
for receta in recetas_cargadas:
  receta = agregar_resumen_a_receta(receta)

A modo de ejemplo se muestra la receta 'Pollo crujiente con salsa césar', con el resumen incorporado

In [ ]:
recetas_cargadas[1000]

{'titulo': 'Pollo crujiente con salsa césar',
 'url': 'https://recetasdecocina.elmundo.es/2012/08/pollo-crujiente-con-salsa-cesar.html',
 'ingredientes': ['frutos secos',
  'solomillo de pollo',
  'sal',
  'pimienta',
  'huevo',
  'aceite'],
 'pasos': ['Ponemos los frutos secos dentro de un paño limpio de cocina y machamos hasta dejarlo en trocitos pequeños.',
  'Batimos un huevo y rebozamos los solomillos de pollo, previamente salpimentados.',
  'Los freímos en abundante aceite caliente y servimos acompañándolos con la salsa césar.'],
 'resumen': 'Ponemos los frutos secos dentro de un paño limpio de cocina y machamos hasta dejarlo en trocitos pequeños. Batimos un huevo y rebozamos los solomillos de pollo, previamente salpimentados. Los freímos en abundante aceite caliente y servimos acompañándolos con la salsa césar.'}

El siguiente paso consiste en el procesamiento y normalización del dataset para garantizar su compatibilidad con el pipeline de entrenamiento, que nos permitirá obtener  las etiquetas (ingredientes) y su posición en el texto. Este servirá de input para entrenar el modelo NER.

Para ello, se implementan las siguientes funciones:. Para ello se definen las funciones:

- ***_quitar_tildes***: devuelve el texto  sin tildes/diacríticos y mantiene el resto igual.

- ***_build_norm_index_map***: genera la versión normalizada (minúsculas y sin tildes) de original y un mapa que indica, para cada carácter normalizado, qué índice tenía en el texto original.

- ***_plural_es_basico***: aplica reglas simples de español para obtener el plural de una palabra (z→ces, vocal→+s, resto→+es; s/x suelen quedar igual).

- ***_plural_de_frase***: pluraliza solo la última palabra “de contenido” de la frase (saltándose stopwords finales) usando _plural_es_basico.

- ***asociar_ingredientes_en_resumen***: busca en el 'resumen' de la receta cada ingrediente (y, si procede, su plural), ignorando tildes y con opción de límites de palabra, y añade a la receta la lista deduplicada de ingredientes encontrados y sus spans (start, end, surface). Con ello obtendremos de cada receta los ingredientes correctamente etiquetados y posicionados.

In [ ]:
import re, unicodedata
from typing import Tuple

_STOPWORDS = {"de","del","la","el","las","los","al","y","en","con","para"}

def _quitar_tildes(txt: str) -> str:
    return "".join(c for c in unicodedata.normalize("NFD", txt) if unicodedata.category(c) != "Mn")

def _build_norm_index_map(original: str) -> Tuple[str, List[int]]:
    """
    Devuelve (texto_normalizado, idx_map) donde idx_map[i] es el índice del
    carácter original correspondiente al carácter normalizado i.
    """
    norm_chars, idx_map = [], []
    for i, ch in enumerate(original):
        base = _quitar_tildes(ch).lower()
        for _ in base:
            norm_chars.append(_)
            idx_map.append(i)
    return "".join(norm_chars), idx_map

def _plural_es_basico(pal: str) -> str:
    w = _quitar_tildes(pal.lower())
    if not w:
        return w
    if w.endswith("z"):
        return w[:-1] + "ces"
    if w.endswith(("s","x")):
        return w  # heurística: la mayoría quedan invariables
    if re.search(r"[aeiou]$", w):
        return w + "s"
    return w + "es"

def _plural_de_frase(ing: str) -> str:
    toks = ing.lower().split()
    if not toks:
        return ing.lower()
    idx = len(toks) - 1
    while idx >= 0 and toks[idx] in _STOPWORDS:
        idx -= 1
    if idx < 0:
        return ing.lower()
    toks[idx] = _plural_es_basico(toks[idx])
    return " ".join(toks)

def asociar_ingredientes_en_resumen(
    receta: Dict[str, Any],
    usar_limites_palabra: bool = True,
    detectar_plural: bool = True
) -> Dict[str, Any]:
    """
    Si un ingrediente (en singular) está en receta['resumen'],
    lo asocia. También captura el plural (heurístico) y es insensible a tildes.
    Añade:
      - receta['ingredientes_en_resumen'] (lista deduplicada)
      - receta['spans_resumen'] con {ingredient, start, end, surface}
    Offsets 0-based sobre el resumen original. 'end' exclusivo.
    """
    resumen = (receta.get("resumen") or "")
    ingredientes = receta.get("ingredientes") or []
    if not isinstance(resumen, str) or not isinstance(ingredientes, list):
        receta["ingredientes_en_resumen"] = []
        receta["spans_resumen"] = []
        return receta

    # Texto normalizado + mapa a original (insensible a tildes)
    resumen_norm, idx_map = _build_norm_index_map(resumen)

    presentes: List[str] = []
    spans: List[Dict[str, Any]] = []

    for ing in ingredientes:
        ing_sing = (ing or "").strip()
        if not ing_sing:
            continue

        # variantes: singular + plural (heurístico)
        variantes = [ing_sing]
        if detectar_plural:
            pl = _plural_de_frase(ing_sing)
            if pl and pl != ing_sing:
                variantes.append(pl)

        # buscamos cada variante sobre el texto normalizado
        seen_positions = set()
        for var in variantes:
            var_norm = _quitar_tildes(var).lower()
            if not var_norm:
                continue

            if usar_limites_palabra:
                pat = re.compile(rf"(?<!\w){re.escape(var_norm)}(?!\w)")
                matches = list(pat.finditer(resumen_norm))
                if matches and ing_sing not in presentes:
                    presentes.append(ing_sing)
                for m in matches:
                    s_n, e_n = m.span()
                    s = idx_map[s_n]
                    e = idx_map[e_n - 1] + 1
                    if (s, e) in seen_positions:
                        continue
                    seen_positions.add((s, e))
                    spans.append({
                        "ingredient": ing_sing,
                        "start": s,
                        "end": e,
                        "surface": resumen[s:e]
                    })
            else:
                start_n = 0
                found = False
                while True:
                    idx_n = resumen_norm.find(var_norm, start_n)
                    if idx_n == -1:
                        break
                    s = idx_map[idx_n]
                    e = idx_map[idx_n + len(var_norm) - 1] + 1
                    if (s, e) not in seen_positions:
                        seen_positions.add((s, e))
                        spans.append({
                            "ingredient": ing_sing,
                            "start": s,
                            "end": e,
                            "surface": resumen[s:e]
                        })
                        found = True
                    start_n = idx_n + len(var_norm)
                if found and ing_sing not in presentes:
                    presentes.append(ing_sing)

    # deduplicar presentes manteniendo orden
    seen_ing = set()
    presentes = [x for x in presentes if not (x in seen_ing or seen_ing.add(x))]

    receta["ingredientes_en_resumen"] = presentes
    receta["spans_resumen"] = sorted(spans, key=lambda s: (s["start"], s["end"]))
    return receta


A modo de ejemplo se muestra una entrada de la lista, correspondiente a la receta "Pochas con pollo de corral", una vez normalizada.

In [ ]:
for receta in recetas_cargadas:
  asociar_ingredientes_en_resumen(receta,usar_limites_palabra=True)

In [ ]:
recetas_cargadas[1000]

{'titulo': 'Pollo crujiente con salsa césar',
 'url': 'https://recetasdecocina.elmundo.es/2012/08/pollo-crujiente-con-salsa-cesar.html',
 'ingredientes': ['frutos secos',
  'solomillo de pollo',
  'sal',
  'pimienta',
  'huevo',
  'aceite'],
 'pasos': ['Ponemos los frutos secos dentro de un paño limpio de cocina y machamos hasta dejarlo en trocitos pequeños.',
  'Batimos un huevo y rebozamos los solomillos de pollo, previamente salpimentados.',
  'Los freímos en abundante aceite caliente y servimos acompañándolos con la salsa césar.'],
 'resumen': 'Ponemos los frutos secos dentro de un paño limpio de cocina y machamos hasta dejarlo en trocitos pequeños. Batimos un huevo y rebozamos los solomillos de pollo, previamente salpimentados. Los freímos en abundante aceite caliente y servimos acompañándolos con la salsa césar.',
 'ingredientes_en_resumen': ['frutos secos', 'huevo', 'aceite'],
 'spans_resumen': [{'ingredient': 'frutos secos',
   'start': 12,
   'end': 24,
   'surface': 'frutos s

Dado que no todos los ingredientes presentan una frecuencia de aparición suficiente para garantizar un entrenamiento estadísticamente robusto de sus etiquetas, se establece un umbral mínimo de 50 ocurrencias. Únicamente se considerarán para el entrenamiento aquellos ingredientes cuya frecuencia supere dicho umbral.

Para ello, se implementa la función ***top_ingredientes_frecuentes*** que filtra y retorna el conjunto de ingredientes con más de 50 apariciones en el dataset.

In [ ]:
from collections import Counter
def top_ingredientes_frecuentes(
    recetas: List[Dict[str, Any]],
    umbral: int = 50,
    clave_spans: str = "spans_resumen",
    clave_ing: str = "ingredient"
) -> List[str]:
    """
    Cuenta menciones por ingrediente a partir de 'spans_resumen' y
    devuelve una lista de ingredientes (strings) que superan el umbral.
    - 'umbral' aplica como '>' (estrictamente mayor que).
    """
    c = Counter()
    for r in recetas:
        for s in (r.get(clave_spans) or []):
            ing = s.get(clave_ing)
            if ing:
                c[ing] += 1

    # filtrar > umbral y ordenar desc por conteo, luego alfabético
    filtrados = [(ing, n) for ing, n in c.items() if n > umbral]
    ordenados = sorted(filtrados, key=lambda x: (-x[1], x[0]))
    return [ing for ing, _ in ordenados]

In [ ]:
ingredientes_frecuentes = top_ingredientes_frecuentes(recetas_cargadas)
print(ingredientes_frecuentes)

['sal', 'ajo', 'cebolla', 'patata', 'aceite de oliva', 'huevo', 'tomate', 'pollo', 'harina', 'agua', 'perejil', 'arroz', 'leche', 'mantequilla', 'zanahoria', 'azúcar', 'pimentón', 'aceite', 'puerro', 'pimiento', 'bacalao', 'vino blanco', 'nata', 'pan', 'salmón', 'caldo de pollo', 'cebolleta', 'vinagre', 'caldo', 'pasta', 'queso', 'limón', 'pimienta', 'caldo de pescado', 'merluza', 'calabacín', 'garbanzo', 'atún']


Filtramos las recetas de manera que el span solo se haga con los ingredientes de la lista top_frecuentes. Si una receta no contiene ninguno de esos ingredientes se elimina

Con la función ***filtrar_spans_por_frecuentes***. Filtramos las recetas de manera que el span solo se haga con los ingredientes de la lista top_frecuentes. Si una receta no contiene ninguno de esos ingredientes se elimina del corpus de entrenamiento.

In [ ]:
from collections.abc import Iterable
from typing import List, Dict, Any

def filtrar_spans_por_frecuentes(
    recetas: List[Dict[str, Any]],
    ingredientes_frecuentes: Iterable,  # ["tomate", ...] o [("tomate", 123), ...]
    clave_spans: str = "spans_resumen",
    clave_ing: str = "ingredient",
    actualizar_ingredientes_en_resumen: bool = True,
    eliminar_recetas_sin_spans: bool = False,
    eliminar_si_ingredientes_en_resumen_vacio: bool = True
) -> List[Dict[str, Any]]:
    """
    Mantiene en cada receta solo los spans cuyo 'ingredient' esté en 'ingredientes_frecuentes'.
    - Si 'actualizar_ingredientes_en_resumen' es True, recalcula esa lista con los spans filtrados.
    - Si 'eliminar_recetas_sin_spans' es True, descarta recetas sin spans tras filtrar.
    - Si 'eliminar_si_ingredientes_en_resumen_vacio' es True, descarta recetas cuyo
      'ingredientes_en_resumen' quede vacío o nulo tras el filtrado.
    Devuelve una **nueva lista** (no modifica la original).
    """
    # Normaliza lista de permitidos a set de strings
    allowed = set()
    for x in ingredientes_frecuentes or []:
        name = x[0] if isinstance(x, (list, tuple)) else x
        if isinstance(name, str) and name.strip():
            allowed.add(name.strip())

    nuevas_recetas: List[Dict[str, Any]] = []
    for r in recetas:
        spans = r.get(clave_spans) or []
        filtrados = [s for s in spans
                     if isinstance(s, dict) and s.get(clave_ing) in allowed]

        # Clona receta y escribe spans filtrados
        r2 = dict(r)
        r2[clave_spans] = sorted(
            filtrados, key=lambda s: (s.get("start", 0), s.get("end", 0))
        )

        # Actualiza lista de ingredientes_en_resumen a partir de los spans filtrados
        if actualizar_ingredientes_en_resumen:
            vistos = set()
            ing_list = []
            for s in r2[clave_spans]:
                lab = s.get(clave_ing)
                if lab and lab not in vistos:
                    vistos.add(lab)
                    ing_list.append(lab)
            r2["ingredientes_en_resumen"] = ing_list

        # Eliminaciones opcionales
        if eliminar_recetas_sin_spans and not r2[clave_spans]:
            continue

        if eliminar_si_ingredientes_en_resumen_vacio:
            ing_res = r2.get("ingredientes_en_resumen")
            if not ing_res:  # None, [], "", etc. se consideran vacíos
                continue

        nuevas_recetas.append(r2)

    return nuevas_recetas

In [ ]:
recetas_filtradas = filtrar_spans_por_frecuentes(recetas_cargadas, ingredientes_frecuentes=ingredientes_frecuentes)

El tamaño del dataset apenas se ve reducido tras preprocesarlo.


In [ ]:
len(recetas_filtradas)

1104

Para terminar con el modulo de preprocesamiento,reestructuramos los diccionarios para adaptarlos al formato requerido por los modelos de NER multiclase. Para ello utilizamos las siguientes funciones:

- ***_resolve_overlaps_longest:*** ordena los spans por inicio y longitud (más largos primero por posición) y elimina solapamientos quedándose con la secuencia no solapada que prioriza el span más largo.

- ***recetas_a_ner_multiclase:*** transforma las recetas en ejemplos de NER multiclase usando cada ingredient como etiqueta, opcionalmente resuelve solapes, valida offsets, y devuelve la lista de ejemplos {id, text, entities} junto con el conjunto ordenado de etiquetas

In [ ]:
def _resolve_overlaps_longest(spans):
    if not spans: return []
    spans = sorted(spans, key=lambda s: (s["start"], -(s["end"]-s["start"])))
    out = []
    for s in spans:
        if not out or s["start"] >= out[-1]["end"]:
            out.append(s)
        # si solapa, se queda el previo (más largo, ya que ordenamos por longitud desc)
    return out

def recetas_a_ner_multiclase(recetas, resolver_solapes=True):
    """
    Usa el campo 'ingredient' de cada span como label (ya normalizado).
    Formato de salida: [{id, text, entities:[{start,end,label}, ...]}, ...]
    """
    ejemplos = []
    label_set = set()

    for i, r in enumerate(recetas, 1):
        text = r.get("resumen") or ""
        spans = r.get("spans_resumen") or []
        use = _resolve_overlaps_longest(spans) if resolver_solapes else sorted(spans, key=lambda s:(s["start"], s["end"]))

        entities = []
        for s in use:
            start, end = int(s["start"]), int(s["end"])
            # validación suave de offsets
            surface = text[start:end]
            if "surface" in s and s["surface"] != surface:
                # si no cuadra, puedes loguear o continuar
                pass
            lab = s.get("ingredient", "")
            if not lab:
                continue
            entities.append({"start": start, "end": end, "label": lab})
            label_set.add(lab)

        ejemplos.append({
            "id": r.get("id", f"receta_{i:05d}"),
            "text": text,
            "entities": entities
        })

    return ejemplos, sorted(label_set)

Guardamos el dataset de entrenamiento del modelo NER.

In [ ]:
train_data, labels = recetas_a_ner_multiclase(recetas_filtradas)

In [ ]:
#guardamos el dataset que servirá de entrenamiento (+ validacion + test) del modelo NER
Path("train_data.json").write_text(
    json.dumps(train_data, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

1296821

In [ ]:
# Leerlo de vuelta para chequear que todo esta bien
contenido = Path("train_data.json").read_text(encoding="utf-8")
data_train = json.loads(contenido)

A modo de ejemplo se muestra uan entrada del dataset de entrenamiento.

In [ ]:
data_train[922]

{'id': 'receta_00923',
 'text': 'La salsa es tan sencilla como poner todo los ingredientes a fuego lento y dejar que se vayan ligando. Pasados unos 5 minutos añadimos el pollo cortado en dados. Seguidamente las verduras cortadas como veis en la foto. Dejamos cocinar durante unos 10-15 minutos a fuego medio y listo. Cocemos arroz tailandes largo para acompañar el plato, fundamental porque pica mucho y os calmará! jejeje.',
 'entities': [{'start': 137, 'end': 142, 'label': 'pollo'},
  {'start': 292, 'end': 297, 'label': 'arroz'}]}